Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./whiskey-reviews-dspt4/train.csv')
test = pd.read_csv('./whiskey-reviews-dspt4/test.csv')
print(train.shape, test.shape)

(4087, 3) (1022, 2)


In [3]:
train.head()

id                                        description  ratingCategory
0  1321  \nSometimes, when whisky is batched, a few lef...               1
1  3861  \nAn uncommon exclusive bottling of a 6 year o...               0
2   655  \nThis release is a port version of Amrut’s In...               1
3   555  \nThis 41 year old single cask was aged in a s...               1
4  1965  \nQuite herbal on the nose, with aromas of dri...               1

In [2]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [6]:
# Read a few reviews from the "Excellent" category
pd.set_option('display.max_colwidth', 0)
train[train.ratingCategory == 0].sample(3)

id  \
2089  4949   
3857  4458   
245   4268   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             description  \
2089  \nPlainly states it’s “aged four months in a small oak barrel.” Tame nose; working hard to pull out anything other than alcohol heat, some fresh-cut wet wood, and wet cereal. Quite hot, with lots of wet oak sawdust and sweet cinnamon that rocket right into the finish. Reminds me of a young teenager, running around, shouting at inappropriate moments, and very excited. Too young and hot right now. Put it back in the barrel; maybe a larger one. Price is per 375 ml.                   
3857  \nWilliam Grant’s answer to Drambuie is a much less sweet and higher proof Irish whiskey-based liqueur. Monster caramel leaps out of the glass with an undercurrent of cola. Caramel leads to chocolate on the palate before malt and grain whiskey undertones emerge. Mid-palate is a little hot, but it helps cut through the underlying sweetness and also features some botanicals like juniper and black pepper. Clearly a mixing spirit and probably best thrown into a cola or ginger ale.    
245   \nHunter Laing blended this grand Highland whisky: an 85% malt-driven recipe of more mature stock. I liked its aromas of dripping caramels, dry peats, soft hay, and faint pepper balancing the resilient smoky character. There’s the juiciness of mandarin oranges, too. The blend is smooth and well-textured, beginning with sugared orange, malt, vanilla, and toasted spices which glide into clove and black pepper. The drying finish has spicy pepper, ground ginger, and lasting citrus.   

      ratingCategory  
2089  0               
3857  0               
245   0

In [8]:
# Read a few reviews from the "Poor" category
train[train.ratingCategory == 2].sample(3)

id  \
1252  5053   
303   5049   
1247  5047   

                                                                                                                                                                                                                                                                                                                                                                                             description  \
1252  \nSo that’s what wheat smells like: not much. The aromas here seem to be mostly yeast-derived, with a faint ripe fruitiness (undefined: peach, apple?) and alcohol heat, and a touch of wheat-origin sweet grass. Wheat’s in the mouth, like chewing fresh grain with some water and alcohol. It’s all there, and the finish is sweet, and it’s clean, but…that’s about all. White bread whiskey.    
303   \nIts age certainly shows, with a good dose of leather, tobacco, and barrel char. Sweet notes of caramel, toffee, and candied fruit try to rescue it, but ultimately fails toward the finish, leaving my mouth parched, leathery, and craving a glass of water (or a beer!). A bourbon with individuality, but it pains me to think what this could have been (or once was).                         
1247  \nFreshly cut oak and chewing tobacco to start, followed by hints of green apple, wood chips, and a warm, wet sharpening stone. On the palate, the wood heightens and becomes extremely bitter. This whiskey’s saving grace is that it’s extremely palatable in cocktails.                                                                                                                           

      ratingCategory  
1252  2               
303   2               
1247  2

### Split the Training Set into Train/Validation (Optional)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train['description'], 
                                                    train['ratingCategory'], 
                                                    test_size=0.2, 
                                                    stratify=train['ratingCategory'],
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3269,) (818,) (3269,) (818,)


### Define Pipeline Components

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [12]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:   11.5s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [13]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [14]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [15]:
# Make Sure the Category is an Integer
submission.head()

id  ratingCategory
0  3461               1
1  2604               1
2  3341               1
3  3764               1
4  2306               1

In [16]:
subNumber = 0

In [17]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [18]:
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD

# Use parameters from RandomSearch in previous section
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=2, 
                       max_df=0.7,
                       max_features=10000)

# Add dimensionality reduction
svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)

# Use parameters from RandomSearch in previous section
rfc = RandomForestClassifier(n_estimators=500, 
                             random_state=42)

pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # Truncated SVD Dimensionality Reduction
    ('clf', rfc)         # RandomForest Classifier
])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [24]:
parameters = {
#     'lsi__svd__n_components': [10,100,250],
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   18.0s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.7,
                                                        max_features=10000,
                                                        min_df=2,
                                                        ngram_range=(1, 2),
                                                        n

### Make a Submission File

In [25]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [26]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [27]:
# Make Sure the Category is an Integer
submission.head()

id  ratingCategory
0  3461               1
1  2604               1
2  3341               1
3  3764               1
4  2306               1

In [28]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [29]:
# Apply to your Dataset
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

param_dist = {
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

In [110]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

X = get_word_vectors(X_train)
len(X) == len(X_train)

X2 = get_word_vectors(train['description'])
len(X2) == len(train['description'])

True

In [169]:
# rfc.fit(X2, train['ratingCategory'])

In [170]:
rfc.fit(X, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [147]:
# X_test_embed = get_word_vectors(X_test)
# This didn't work

In [151]:
X_test_embed2 = get_word_vectors(test['description'])

In [150]:
# This is the source of the main issue I was having

X_test.shape, test['description'].shape

((818,), (1022,))

### Make a Submission File

In [164]:
# from sklearn.metrics import accuracy_score

# # Evaluate on test data
y_pred = rfc.predict(X_test_embed2)
# accuracy_score(test['ratingCategory'], y_pred)

# Accuracy_score would be 0.0 since test labels aren't provided

In [165]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':y_pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [166]:
# Make Sure the Category is an Integer
submission.head()

id  ratingCategory
0  3461               1
1  2604               1
2  3341               1
3  3764               1
4  2306               1

In [167]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?